<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/LagrangePoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [154]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root
import jax.numpy as jnp
import jax
from jax.config import config
config.update("jax_enable_x64", True)
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [149]:

m1, m2 = (50,30)
r1=np.array([-5.,0,0])
r2=np.array([5.,0,0])
rc = (m1*r1+m2*r2)/(m1+m2)
w = ((m1+m2)/(np.linalg.norm(r2-r1))**3)**(0.5) * np.array([0,0,1.])

#set origin relative to center of mass
r1=r1-rc
r2=r2-rc
r1,r2

(array([-3.75,  0.  ,  0.  ]), array([6.25, 0.  , 0.  ]))

In [165]:
def f(r):
    return -m1/((jnp.linalg.norm(r-r1))**3)*(r-r1) - m2/((jnp.linalg.norm(r-r2))**3)*(r-r2) - jnp.cross(w, jnp.cross(w,r))

In [263]:
res=root(f,[-10,0,0])
res

    fjac: array([[-1.,  0.,  0.],
       [ 0., -1.,  0.],
       [ 0.,  0., -1.]])
     fun: array([ 3.41948692e-14, -0.00000000e+00,  0.00000000e+00])
 message: 'The solution converged.'
    nfev: 10
     qtf: array([-6.22823637e-09,  0.00000000e+00,  0.00000000e+00])
       r: array([-0.30342957,  0.        ,  0.        ,  0.13179135,  0.        ,
        0.21179135])
  status: 1
 success: True
       x: array([-11.52524009,   0.        ,   0.        ])

In [228]:
res.x

array([1.7734263, 0.       , 0.       ])

In [176]:
def rhs(t,vec):
    v,r = jnp.split(vec,2)
    dv=-m1/((jnp.linalg.norm(r-r1))**3)*(r-r1) - m2/((jnp.linalg.norm(r-r2))**3)*(r-r2) - jnp.cross(w, jnp.cross(w,r)) - 2*jnp.cross(w,v)
    dr = v  
    return jnp.concatenate([dv,dr])

jax_rhs = jax.jit(rhs)
jax_rhs_jac = jax.jit(jax.jacobian(rhs,(1)))

In [285]:
# vr0 = np.array([0.0,-0.0, 0.0, 12.38038367, 0.0, 0.])     #L2
# vr0 = np.array([0.0,-0.0, 0.0, 1.77342625, 0.0, 0.])      #L1
# vr0 = np.array([0.0, 0.0, 0.0, 1.25, 8.66025404, 0.])     #L4
# vr0 = np.array([0.0, 0.0, 0.0, 1.25, -8.66025404, 0.])    #L5
# vr0 = np.array([0.0, 0.0, 0.0, -11.52524009, 0., 0.])     #L1
vr0 = np.array([-1., -1., 0, 0, 0, 0]) #start from center of mass
tend=20
res=solve_ivp(jax_rhs,(0,tend),vr0,method='Radau',dense_output=True, jac=jax_rhs_jac)

In [286]:
def equispaced_t(tend, dmax=0.005):
    tplot=[]
    t=0
    v0,r0 = np.split(vr0,2)
    tinc=1
    while t<tend:
        while True:
            v,r=np.split(res.sol(t+tinc),2)
            dist = np.linalg.norm(r-r0)
            if dist>dmax:
                tinc=tinc/2
            elif dist<dmax/4:
                tinc=tinc*1.5
            else:
                tplot.append(t+tinc)
                t=t+tinc
                r0=r
                break
    return np.array(tplot)


In [287]:
# tplot = np.linspace(0,tend,1000)
tplot = equispaced_t(tend)

In [288]:

dx,dy,dz,x,y,z=res.sol(tplot)
fig=make_subplots()
fig.add_trace(go.Scatter(x=x,y=y,mode='lines'))
fig.update_layout(width=800,height=800,template='plotly_dark')

In [33]:
G=6.67e-11
Mmoon = 7.35e22
Mearth = 5.97e24
Rmoon_earth = 400e6

In [34]:
omega = (G*(Mmoon+Mearth)/Rmoon_earth**3)**(0.5)

In [38]:
2*np.pi/omega/3600/24

28.976700546754795